먼저, 내가 open3d로 만든 FPFH디스크립터가 어느정도 잘 동작하는지 확인하고 학습해야 할 듯.
내가 잘못 만들어서 문제가 된 것일 수 있으니 확인

In [1]:
import os
import sys
import argparse
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import open3d.visualization as vis
import torch
from scipy.spatial.distance import cdist

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
opt = argparse.Namespace(
    fpfh_normal_radiuse = 0.3,
    fpfh_descriptors_radiuse = 1.0,
    seq_list = [0],
    mdgat_path = './KITTI',
    kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
    memory_is_enough = True,
    mutual_check = True,
    transform_opt = 0
)

In [6]:
class mdgat_kitti_data_set():
    def __init__(self, opt):
        self.kitti_path = opt.kitti_path
        self.mdgat_path = opt.mdgat_path
        self.seq_list = opt.seq_list
        self.memory_is_enough = opt.memory_is_enough
        self.mutual_check = opt.mutual_check
        self.transform_opt = opt.transform_opt
        self.fpfh_normal_radiuse = opt.fpfh_normal_radiuse
        self.fpfh_descriptors_radiuse = opt.fpfh_descriptors_radiuse

        self.dataset = []
        self.calib = {}
        self.pose = {}
        self.kp = {}
        self.desc = {}
        self.scores = {}
        self.scan = {}
        self.random_sample_num = 16384 #16384
        self.threshold = 0.5

        self._load_kitti_gt_txt()
        self._load_preprocessed_data()

    def _load_kitti_gt_txt(self):
        '''
        :param txt_root:
        :param seq
        :return: [{anc_idx: *, pos_idx: *, seq: *}]                
        '''
        for seq in self.seq_list:
            with open(os.path.join(self.mdgat_path, 'preprocess-random-full','%02d'%seq, 'groundtruths.txt'), 'r') as f:
                lines_list = f.readlines()
                for i, line_str in enumerate(lines_list):
                    if i == 0:
                        # skip the header line
                        continue
                    line_splitted = line_str.split()
                    anc_idx = int(line_splitted[0])
                    pos_idx = int(line_splitted[1])

                    data = {'seq': seq, 'anc_idx': anc_idx, 'pos_idx': pos_idx}
                    self.dataset.append(data)
    
    def _load_preprocessed_data(self):
        for seq in self.seq_list:
            sequence = '%02d'%seq
            calibpath = os.path.join(self.mdgat_path, 'calib/sequences', sequence, 'calib.txt')
            posepath = os.path.join(self.mdgat_path, 'poses', '%02d.txt'%seq)
            with open(calibpath, 'r') as f:
                for line in f.readlines():
                    _, value = line.split(':', 1)
                    try:
                        calib = np.array([float(x) for x in value.split()])
                    except ValueError:
                        pass
                    calib = np.reshape(calib, (3, 4))    
                    self.calib[sequence] = np.vstack([calib, [0, 0, 0, 1]])
            
            poses = []
            with open(posepath, 'r') as f:
                for line in f.readlines():
                    T_w_cam0 = np.fromstring(line, dtype=float, sep=' ')
                    T_w_cam0 = T_w_cam0.reshape(3, 4)
                    T_w_cam0 = np.vstack((T_w_cam0, [0, 0, 0, 1]))
                    poses.append(T_w_cam0)
                self.pose[sequence] = poses

            '''If memory is enough, load all the data'''
            if self.memory_is_enough:
                kps = []
                scores = []
                desc = []
                folder = os.path.join(self.mdgat_path, 'keypoints/tsf_256_FPFH_16384-512-k1k16-2d-nonoise', sequence)
                folder = os.listdir(folder)   
                folder.sort(key=lambda x:int(x[:-4]))
                for idx in range(len(folder)):
                    file = os.path.join(self.mdgat_path, 'keypoints/tsf_256_FPFH_16384-512-k1k16-2d-nonoise', sequence, folder[idx])
                    if os.path.isfile(file):
                        pc = np.reshape(np.fromfile(file, dtype=np.float32), (-1, 37))
                        ones = np.ones((pc.shape[0], 1))
                        kps.append(np.concatenate((pc[:,:3], ones), axis=1))
                        scores.append(pc[:,3])
                        desc.append(pc[:,4:])
                    else:
                        kps.append([0])

                self.kp[sequence] = kps
                self.desc[sequence] = desc
                self.scores[sequence] = scores

    def _get_kitti_data(self, sequence, index_in_seq):
        pc_file = os.path.join('/media/vision/Seagate/DataSets/kitti/dataset/sequences', sequence, "velodyne" ,'%06d.bin' % index_in_seq)
        pc = np.fromfile(pc_file, dtype=np.float32)
        pc = pc.reshape((-1, 4))

        ones = np.ones((pc.shape[0], 1))
        pc = np.concatenate((pc[:,:3], ones), axis=1)
        # pc = pc[np.random.choice(pc.shape[0], self.random_sample_num, replace=False), :]
        pc = torch.tensor(pc, dtype=torch.double)
        # pc = pc.reshape((-1, 8))
        return pc
    
    def _get_kitti_poseset(self, idx):
        data = self.dataset[idx]
        sequence = '%02d'%data['seq']
        anc_idx = data['anc_idx']
        pos_idx = data['pos_idx']
        pc0 = self._get_kitti_data(sequence, anc_idx)
        pc1 = self._get_kitti_data(sequence, pos_idx)
        return pc0, pc1
    
    def __len__(self):
        return len(self.dataset)
    
    def comute_FPFH(self, pc, kp):
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(pc[:, :3])

        kp_num = kp.shape[0]
        pcd_kp = o3d.geometry.PointCloud()
        pcd_kp.points = o3d.utility.Vector3dVector(kp[:, :3])
        pcd += pcd_kp

        pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=self.fpfh_normal_radiuse, max_nn=30))

        pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=self.fpfh_normal_radiuse, max_nn=100))
        fpfh_desc = np.transpose(pcd_fpfh.data)[-kp_num:]
        fpfh_desc = torch.tensor(fpfh_desc, dtype=torch.double)
        return fpfh_desc
    
    def get_data(self, seq, idx):
        data = self.dataset[idx]
        sequence = '%02d'%seq

        pose = torch.tensor(self.pose[sequence][idx], dtype=torch.double)
        kp = torch.tensor(self.kp[sequence][idx], dtype=torch.double)
        desc = torch.tensor(self.desc[sequence][idx], dtype=torch.double)
        score = torch.tensor(self.scores[sequence][idx], dtype=torch.double)
        pc = self._get_kitti_data(sequence, idx)
        # fpfh_desc = self.comute_FPFH(pc, kp)

        T_cam0_velo = self.calib[sequence]
        T_cam0_velo = torch.tensor(T_cam0_velo, dtype=torch.double)

        '''transform point cloud from cam0 to LiDAR'''
        kpw = torch.einsum('ki,ij,jm->mk', pose, T_cam0_velo, kp.T)
        pcw = torch.einsum('ki,ij,jm->mk', pose, T_cam0_velo, pc.T)

        top_k = 10
        top_scores, top_indices = torch.topk(score, k=top_k)
        kp = kp[top_indices]
        kpw = kpw[top_indices]
        desc = desc[top_indices]

        norm = np.linalg.norm(desc, axis=1)
        norm = norm.reshape(len(kp), 1)
        desc = np.where(norm != 0, np.multiply(desc, 1/norm), 0)

        # fpfh_norm = np.linalg.norm(fpfh_desc, axis=1)
        # fpfh_norm = fpfh_norm.reshape(len(kp), 1)
        # fpfh_desc = np.where(fpfh_norm != 0, np.multiply(fpfh_desc, 1/fpfh_norm), 0)

        return {
            # 'skip': False,
            'keypoints': kp,
            'keypointsw': kpw,
            'descriptors': desc,
            # 'fpfh_descriptors': fpfh_desc,
            'scores': score,
            'sequence': sequence,
            'idx': idx,
            'pose': pose,
            'T_cam0_velo': T_cam0_velo,
            'cloud': pc,
            'cloudw': pcw,
        } 

    def __getitem__(self, idx):
        data = self.dataset[idx]
        sequence = '%02d'%data['seq']
        anc_idx = data['anc_idx']
        pos_idx = data['pos_idx']
        anc_pc, pos_pc = self._get_kitti_poseset(idx)
        return anc_pc, pos_pc


순서대로 모든 pcw를 합쳐 visualization을 해보자.

In [8]:
data_set = mdgat_kitti_data_set(opt=opt)
world_pcd = o3d.geometry.PointCloud()
for i in range(0, len(data_set)):
    data = data_set.get_data(0, i)
    pcw = data['cloudw']
    pcdw = o3d.geometry.PointCloud()
    pcdw.points = o3d.utility.Vector3dVector(pcw[:, :3])
    pcdw.voxel_down_sample(voxel_size=0.3)
    world_pcd += pcdw
    if i % 10 == 0:
        print("process %d/%d"%(i, len(data_set)))

vis.draw_geometries([world_pcd])

process 70/4539


역시 딱 맞지 않는다. 

In [8]:
data_set = mdgat_kitti_data_set(opt=opt)
world_kpd = o3d.geometry.PointCloud()
for i in range(0, len(data_set)):
    data = data_set.get_data(0, i)
    kpw = data['keypointsw']
    kp_pcdw = o3d.geometry.PointCloud()
    kp_pcdw.points = o3d.utility.Vector3dVector(kpw[:, :3])
    kp_pcdw.voxel_down_sample(voxel_size=0.3)
    world_kpd += kp_pcdw
    if i % 10 == 0:
        print("process %d/%d"%(i, len(data_set)))

vis.draw_geometries([world_kpd])

process 0/4539
process 10/4539
process 20/4539
process 30/4539
process 40/4539
process 50/4539
process 60/4539
process 70/4539
process 80/4539
process 90/4539
process 100/4539
process 110/4539
process 120/4539
process 130/4539
process 140/4539
process 150/4539
process 160/4539
process 170/4539
process 180/4539
process 190/4539
process 200/4539
process 210/4539
process 220/4539
process 230/4539
process 240/4539
process 250/4539
process 260/4539
process 270/4539
process 280/4539
process 290/4539
process 300/4539
process 310/4539
process 320/4539
process 330/4539
process 340/4539
process 350/4539
process 360/4539
process 370/4539
process 380/4539
process 390/4539
process 400/4539
process 410/4539
process 420/4539
process 430/4539
process 440/4539
process 450/4539
process 460/4539
process 470/4539
process 480/4539
process 490/4539
process 500/4539
process 510/4539
process 520/4539
process 530/4539
process 540/4539
process 550/4539
process 560/4539
process 570/4539
process 580/4539
process 

In [ ]:
# 비주얼라이제이션
data_set = mdgat_kitti_data_set(opt=opt)
data = data_set.get_data(1100)
coord = 1
if coord == 0:
    pc0 = data['cloud0']
    kp0 = data['keypoints0']
    pc1 = data['cloud1']
    kp1 = data['keypoints1']
else:
    pc0 = data['cloudw0']
    kp0 = data['keypointsw0']
    pc1 = data['cloudw1']
    kp1 = data['keypointsw1']

# Create Open3D point cloud objects
pcd0 = o3d.geometry.PointCloud()
pcd0.points = o3d.utility.Vector3dVector(pc0[:, :3])
pcd0.colors = o3d.utility.Vector3dVector(np.ones((pc0.shape[0], 3)) * [0.5, 0.5, 0.5])

pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(pc1[:, :3])
pcd1.colors = o3d.utility.Vector3dVector(np.ones((pc1.shape[0], 3)) * [0.2, 0.2, 0.2])

kpcd0 = o3d.geometry.PointCloud()
kpcd0.points = o3d.utility.Vector3dVector(kp0[:, :3])
kpcd0.colors = o3d.utility.Vector3dVector(np.ones((kp0.shape[0], 3)) * [1, 0, 0])

kpcd1 = o3d.geometry.PointCloud()
kpcd1.points = o3d.utility.Vector3dVector(kp1[:, :3])
kpcd1.colors = o3d.utility.Vector3dVector(np.ones((kp1.shape[0], 3)) * [0, 1, 0])

# Visualize the point clouds
vis.draw_geometries([pcd0, kpcd0, pcd1, kpcd1])

만들어진 fpfh값과 precomputed된 fpfh값을 비교해서 어떤 세팅값이 가장 유사한지 찾아보자.

In [ ]:
test_fpfh_normal_radiuses = [0.2, 0.5, 0.8, 1.0, 1.3, 1.5, 1.7]

# test_fpfh_descriptors_radiuses = [0.7 ,1.0, 1.5, 2.0, 2.5]
test_fpfh_descriptors_radiuses = [1.5]

matched_points_rank_mean = np.zeros((len(test_fpfh_normal_radiuses), len(test_fpfh_descriptors_radiuses)))
matched_pointe_dists_mean = np.zeros((len(test_fpfh_normal_radiuses), len(test_fpfh_descriptors_radiuses)))

for j, fpfh_normal_radiuse in enumerate(test_fpfh_normal_radiuses):
    for k, fpfh_descriptors_radiuses in enumerate(test_fpfh_descriptors_radiuses):
        opt = argparse.Namespace(
            fpfh_normal_radiuse = fpfh_normal_radiuse,
            fpfh_descriptors_radiuse = fpfh_descriptors_radiuses,
            seq_list = [0, 2],
            mdgat_path = './KITTI',
            kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
            memory_is_enough = True,
            mutual_check = True,
            transform_opt = 0
        )
        data_set = mdgat_kitti_data_set(opt=opt)

        n = []

        rank_sum = 0
        dist_sum = 0
        ranks = []

        for i in range(100, len(data_set), 500):
            data = data_set.get_data(i)
            match0 = data['gt_matches0']
            desc0 = data['fpfh_descriptors0']
            desc1 = data['fpfh_descriptors1']

            nn = 0
            for idx, matched_idx in enumerate(match0):
                nn += 1
                
                if matched_idx != -1:
                    distance = np.linalg.norm(desc0 - desc1[matched_idx], axis=1)
                    dist_sum += distance[idx]
                    rank_sum += np.argsort(distance).tolist().index(idx)
                    ranks.append(np.argsort(distance).tolist().index(idx))
            n.append(nn)

        matched_points_rank_mean[j,k] = rank_sum / sum(n)
        matched_pointe_dists_mean[j,k] = dist_sum / sum(n)

        print('radius: (', fpfh_normal_radiuse, ', ', fpfh_descriptors_radiuses, ') ', 'rank mean: ', matched_points_rank_mean[j,k], 'dist mean: ', matched_pointe_dists_mean[j,k])
        print('ranks: ', ranks)

만들어진 fpfh가 얼마나 잘 describe 하는지 최적의 세팅값을 찾아보자.

In [ ]:
test_fpfh_normal_radiuses = [0.05, 0.1, 0.2, 0.3, 0.5, 0.8, 1.0, 1.3, 1.5]
test_fpfh_descriptors_radiuses = [0.3, 0.5, 0.7 ,1.0, 1.5, 2.0]

matched_pointe_dists_mean = np.zeros((len(test_fpfh_normal_radiuses), len(test_fpfh_descriptors_radiuses)))

for j, fpfh_normal_radiuse in enumerate(test_fpfh_normal_radiuses):
    for k, fpfh_descriptors_radiuses in enumerate(test_fpfh_descriptors_radiuses):
        opt = argparse.Namespace(
            fpfh_normal_radiuse = fpfh_normal_radiuse,
            fpfh_descriptors_radiuse = fpfh_descriptors_radiuses,
            seq_list = [0, 2],
            mdgat_path = './KITTI',
            kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
            memory_is_enough = True,
            mutual_check = True,
            transform_opt = 0
        )
        data_set = mdgat_kitti_data_set(opt=opt)

        n = []

        rank_sum = 0
        dist_sum = 0

        for i in range(100, len(data_set), 500):
            data = data_set.get_data(i)
            match0 = data['gt_matches0']
            desc0 = data['fpfh_descriptors0']
            desc1 = data['descriptors0']

            nn = 0
            
            for idx in range(len(desc0)):
                nn += 1
                distance = np.linalg.norm(desc0[idx] - desc1[idx])
                dist_sum += distance
            n.append(nn)

        matched_pointe_dists_mean[j,k] = dist_sum / sum(n)

        print('radius: (', fpfh_normal_radiuse, ', ', fpfh_descriptors_radiuses, ') ', 'dist mean: ', matched_pointe_dists_mean[j,k])

In [ ]:
data = data_set.get_data(1100)
# print(data['rep'])
match0 = data['gt_matches0']
match1 = data['gt_matches1']
# desc00 = data['descriptors0']
# desc1 = data['descriptors1']
desc0 = data['fpfh_descriptors0']
desc1 = data['fpfh_descriptors1']
kp0 = data['keypointsw0']
kp1 = data['keypointsw1']
for idx, matched_idx in enumerate(match0):
    if matched_idx != -1:
        print('matched_idx: ', idx, ' & ', matched_idx, ' || distance: ', np.linalg.norm(kp0[idx] - kp1[matched_idx]))
        distance = np.linalg.norm(desc0 - desc1[matched_idx], axis=1)
        print('distance with matched points: ', distance[idx], ' || similarity rank: ', np.argsort(distance).tolist().index(idx), '/', len(distance))
        print('min distance: ', np.min(distance))
        print('mean distance: ', np.mean(distance))
        print('----------------------------------')